In [1]:
# Adapted from spectral_weight_lanczos.ipynb
# Deleted some useless lines
# This program only concerns 1A1 channel's lowest peak 
# and mainly want to explore its dispersion with k in high symmetry points

%pylab inline
import math
import numpy as np
from scipy.sparse.linalg import inv
#from numpy.linalg import inv
import scipy.sparse as sps
import scipy.sparse.linalg
from scipy import integrate
import sys
import matplotlib.pyplot as plt
sys.path.append('../../src/')
from pylab import *

import parameters as pam
import lattice as lat
import variational_space as vs
import hamiltonian as ham
import basis_change as basis
import lanczos

M_PI = math.pi
Mc  = pam.Mc
print 'Mc=',Mc

w_vals = pam.w_vals
Norb = pam.Norb
eta = pam.eta
ed  = pam.ed
ep  = pam.ep
project = pam.if_project_out_two_holes_on_different_Cu
Nk = pam.Nk
    
if Norb==3 or Norb==7:
    tpd = pam.tpd
    tpp = pam.tpp
elif Norb==9:
    pds = pam.pds
    pdp = pam.pdp
    pps = pam.pps
    ppp = pam.ppp
    
Upp = pam.Upp
if Norb==3:
    Udd = pam.Udd
elif Norb==7 or Norb==9:
    A = pam.A
    B = pam.B
    C = pam.C
                    
Ms = ['-b','-r','-g','-m','-c','-k','-y','--b','--r','--g','--m','--c','--k','--y',\
      '-.b','-.r','-.g','-.m','-.c','-.k','-.y',':b',':r',':g',':m',':c',':k',':y']
##########################################################################
def writefile(fname,kxs,kys, w_max_k, Aw_max_k):
    f = open('./data_1A1_lowest_peak_dispersion/'+fname,'w',1) 
    f.write('kx   ky   E_lowest_peak   weight_lowest_peak\n')
    
    for i in xrange(0,len(w_max_k)):
        f.write('{:.6e}\t{:.6e}\t{:.6e}\t{:.6e}\n'.format(kxs[i],kys[i],w_max_k[i],Aw_max_k[i]))
        
def getAw(matrix,index,VS,w_vals):  
    # set up Lanczos solver
    dim  = VS.dim
    scratch = np.empty(dim, dtype = complex)
    Phi0 = np.zeros(dim, dtype = complex)
    Phi0[index] = 1.0
    solver = lanczos.LanczosSolver(maxiter = pam.Lanczos_maxiter, 
                                   precision = 1e-12, 
                                   cond = 'UPTOMAX', 
                                   eps = 1e-8)
    solver.first_pass(x0 = Phi0, scratch = scratch, H = matrix)
    V, D = solver.lanczos_diag_T()

    # D[0,:] is the eigenvector for lowest eigenvalue
    tab = np.abs(D[0,:])**2

    Aw = np.zeros(len(w_vals))
    for n in range(len(V)):
        Aw += tab[n] * eta / M_PI * ( (w_vals - V[n])**2 + eta**2)**(-1)
        
    w_peak, weight = getAw_peak_pos_weight(Aw, D, tab)
    return Aw, w_peak, weight

def getAw_peak_pos_weight(Aw, D, tab):  
    '''
    find the position and weight of lowest peak of Aw
    used to explore their dispersion in the high symmetry points in BZ
    '''    
    w_idx = np.argmax(Aw)
    w_peak = w_vals[w_idx]
    
    # find the area below the whole peak, namely the peak weight
    # ==========================================================
    # 1. first find the peak's w-range: [w_min, w_max]
    wid = w_idx
    while Aw[wid]>1.e-3:
        #print w_vals[wid], Aw[wid]
        if Aw[wid-1]>Aw[wid]:
            break
        wid -= 1
    w_min = wid
    
    wid = w_idx
    while Aw[wid]>1.e-3:
        #print w_vals[wid], Aw[wid]
        if Aw[wid+1]>Aw[wid]:
            break
        wid += 1
    w_max = wid
    print 'peak w-range = [', w_vals[w_min], w_vals[w_max], ']'
    
    # 2. Simpson's rule
    weight = integrate.simps(Aw[w_min:w_max], w_vals[w_min:w_max])
    print 'peak, weight = ', w_peak, '  ', weight
                          
    '''
    # find the eigenvalue D[n] nearest to w_peak so that its index n
    # leads to weight = tab[n]; Note that this weight is actually for 
    # the single peak instead of the area below the whole peak
    tmp = []
    for n in range(len(D)):
        tmp.append(abs(D[n]-w_peak))
        
    idx = tmp.index(min(tmp))
    weight = tab[idx]
    assert(weight>=0.0 and weight<=1.0)
    '''
    return w_peak, weight

def plot_atomic_multiplet_peaks(data_for_maxval):
    maxval = max(data_for_maxval)
    yy = [0,maxval]
    xx = [pam.E_1S,pam.E_1S]
    plt.plot(xx, yy,'--k', linewidth=0.5)
    #text(pam.E_1S-0.2, 10.2, 'E_1S', fontsize=5)
    xx = [pam.E_1G,pam.E_1G]
    plt.plot(xx, yy,'--k', linewidth=0.5)
    #text(pam.E_1G-0.2, 10.5, 'E_1G', fontsize=5)
    xx = [pam.E_1D,pam.E_1D]
    plt.plot(xx, yy,'--k', linewidth=0.5)
    #text(pam.E_1D-0.2, 10.8, 'E_1D', fontsize=5)
    xx = [pam.E_3P,pam.E_3P]
    plt.plot(xx, yy,'--k', linewidth=0.5)
    #text(pam.E_3P-0.2, 11.1, 'E_3P', fontsize=5)
    xx = [pam.E_3F,pam.E_3F]
    plt.plot(xx, yy,'--k', linewidth=0.5)
    #text(pam.E_3F-0.2, 11.4, 'E_3F', fontsize=5)
    
def compute_Aw_main(kx,ky):   
    # set up H0
    VS = vs.VariationalSpace(Mc)
    basis.count_VS(VS)
    phase = ham.create_phase_dict(kx,ky,VS)
    T_pd  = ham.create_tpd_nn_matrix(phase,VS)
    T_pp  = ham.create_tpp_nn_matrix(phase,VS)     
    Esite = ham.create_edep_diag_matrix(VS)
    
    # Note tpd and tpp prefactor are considered in hamiltonian.py
    H0 = T_pd + T_pp + ep*Esite

    d_double, p_double = ham.get_double_occu_list(VS)
    
    # change the basis for d_double states to be singlet/triplet
    #U, S_val, Sz_val = basis.create_singlet_triplet_basis_change_matrix_d_double(VS,d_double)
    U, S_val, Sz_val = basis.create_singlet_triplet_basis_change_matrix(VS)
    U_d = (U.conjugate()).transpose()
    H0_new = U_d.dot(H0.dot(U))

    # check if U if unitary
    #UdU = U_d.dot(U)
    #sh = UdU.shape
    #print sh
    #bb = sps.identity(sh[0], format='coo')
    #tmp = UdU-bb
    #print 'U_d.dot(U)-I = ', tmp
    
    if Norb==3:
        fname = 'ep'+str(ep)+'_tpd'+str(tpd)+'_tpp'+str(tpp)+ \
                '_Udd'+str(Udd)+'_Upp'+str(Upp)+'_kx'+str(kx)+'_ky'+str(ky)+'_Mc'+str(Mc)+'_Norb'+str(Norb)+ \
                '_project'+str(project)
    elif Norb==7:
        fname = 'ep'+str(ep)+'_tpd'+str(tpd)+'_tpp'+str(tpp) \
              +'_A'+str(A)+'_B'+str(B)+'_C'+str(C) \
              +'_Upp'+str(Upp)+'_kx'+str(kx)+'_ky'+str(ky)+'_Mc'+str(Mc)+'_Norb'+str(Norb)+ \
                '_project'+str(project)
    elif Norb==9:
        fname = 'ep'+str(ep)+'_pds'+str(pds)+'_pdp'+str(pdp)+'_pps'+str(pps)+'_ppp'+str(ppp) \
              +'_A'+str(A)+'_B'+str(B)+'_C'+str(C) \
              +'_Upp'+str(Upp)+'_kx'+str(kx)+'_ky'+str(ky)+'_Mc'+str(Mc)+'_Norb'+str(Norb)+ \
                '_project'+str(project)
                
    #======================= opt for Norb=3 or 7, 9 ======================================
    # set up H_int
    if Norb==3:
        Hint, Aw_state_indices = ham.create_interaction_matrix_Norb3(VS,d_double,p_double)
        H = H0_new + Hint
        H.tocsr()
        
        Aw = np.zeros(len(w_vals))
        Nstate = len(Aw_state_indices)
        for j in range(0,Nstate):
            index = Aw_state_indices[j]
            Aw, w_peak, weight = getAw(H,index,VS,w_vals)           

            state = VS.get_state(VS.lookup_tbl[index])
            s1 = state['hole1_spin']
            s2 = state['hole2_spin']
            o1 = state['hole1_orb']
            o2 = state['hole2_orb']
            #subplot(Nstate,1,j+1)
            plt.plot(w_vals, Aw, Ms[j], linewidth=1, label=tuple([s1,o1,s2,o2]))

            maxval = max(Aw)
            #xlim([-5,20])
            ylim([0,maxval])
            ylim([0,0.5])
            ylabel('$A(\omega)$',fontsize=14)
            #text(0.45, 0.1, '(a)', fontsize=16)
            grid('on',linestyle="--", linewidth=0.5, color='black', alpha=0.5)
            legend(loc='best', fontsize=6.5, framealpha=1.0, edgecolor='black')
            if j==0:
                title(fname, fontsize=8)
            if j==Nstate-1:
                xlabel('$\omega$',fontsize=15)

        savefig("./figures_1A1_dispersion/Aw_"+fname+".pdf")
        clf()
        
        return w_peak, weight

    elif Norb==7 or Norb==9:    
        # for multi d-orbital cases, compute A(w) for each symmetry channel
        Aw_dd_total = np.zeros(len(w_vals))
        Aw_d8_total = np.zeros(len(w_vals))
    
        Nsym = len(pam.symmetries)
        for i in range(0,Nsym):
            sym = pam.symmetries[i]
            #print "===================================="
            #print "start computing A_dd(w) for sym", sym
            Hint, dd_state_indices = ham.create_interaction_matrix(VS,sym,d_double,p_double,S_val, Sz_val)
            H = H0_new + Hint
            H.tocsr()
                       
            #dd_states, d8_states, coef_frac_parentage = ham.get_Aw_state(sym)

            # compute G_dd(sym)
            Aw_dd = np.zeros(len(w_vals))
            Aw_d8 = np.zeros(len(w_vals))
        
            for index in dd_state_indices:
                Aw_tmp, w_peak, weight = getAw(H,index,VS,w_vals)
                Aw_dd += Aw_tmp #*coef_frac_parentage[spinorb]
        
            plt.plot(w_vals, Aw_dd, Ms[i], linewidth=1, label=sym)
            
            # plot atomic multiplet peaks
            plot_atomic_multiplet_peaks(Aw_dd)
                
            if i==0:
                title(fname, fontsize=8)
            if i==Nsym-1:
                xlabel('$\omega$',fontsize=15)
                
            #xlim([-5,20])
            maxval = max(Aw_dd)
            #ylim([0,maxval])
            ylim([0,0.5])
            #ylabel('$A(\omega)$',fontsize=17)
            #text(0.45, 0.1, '(a)', fontsize=16)
            grid('on',linestyle="--", linewidth=0.5, color='black', alpha=0.5)
            legend(loc='best', fontsize=9.5, framealpha=1.0, edgecolor='black')
            #yticks(fontsize=12)   
        
        savefig("./figures_1A1_dispersion/Aw_dd_"+fname+"_sym.pdf")
        clf()

        return w_peak, weight
         
##########################################################################
if __name__ == '__main__': 
    if Norb==3:
        fname = 'ep'+str(ep)+'_tpd'+str(tpd)+'_tpp'+str(tpp)+ \
                '_Udd'+str(Udd)+'_Upp'+str(Upp)+'_Mc'+str(Mc)+'_Norb'+str(Norb)+ \
                '_project'+str(project)
    elif Norb==7:
        fname = 'ep'+str(ep)+'_tpd'+str(tpd)+'_tpp'+str(tpp) \
              +'_A'+str(A)+'_B'+str(B)+'_C'+str(C) \
              +'_Upp'+str(Upp)+'_Mc'+str(Mc)+'_Norb'+str(Norb)+ \
                '_project'+str(project)
    elif Norb==9:
        fname = 'ep'+str(ep)+'_pds'+str(pds)+'_pdp'+str(pdp)+'_pps'+str(pps)+'_ppp'+str(ppp) \
              +'_A'+str(A)+'_B'+str(B)+'_C'+str(C) \
              +'_Upp'+str(Upp)+'_Mc'+str(Mc)+'_Norb'+str(Norb)+ \
                '_project'+str(project)

    cnt = 0
    kxs = []
    kys = []
    w_peak_k = []
    weight_k = []
    
    # from (0,0) to (pi,pi)
    for kx in np.arange(0.,M_PI,M_PI/Nk):
        ky = kx
        print 'k = ', kx,ky
        w_peak, weight = compute_Aw_main(kx,ky)
        kxs.append(kx)
        kys.append(ky)
        w_peak_k.append(w_peak)
        weight_k.append(weight)
        cnt += 1

    # from (pi,pi) to (pi,0)
    for ky in np.arange(M_PI,0.,-M_PI/Nk):
        kx = M_PI
        print 'k = ', kx,ky
        w_peak, weight = compute_Aw_main(kx,ky)
        kxs.append(kx)
        kys.append(ky)
        w_peak_k.append(w_peak)
        weight_k.append(weight)
        cnt += 1

    # from (pi,0) to (0,0)
    for kx in np.arange(M_PI,0.,-M_PI/Nk):
        ky = 0.0
        print 'k = ', kx,ky
        w_peak, weight = compute_Aw_main(kx,ky)
        kxs.append(kx)
        kys.append(ky)
        w_peak_k.append(w_peak)
        weight_k.append(weight)
        cnt += 1

    # record data
    writefile(fname,kxs,kys,w_peak_k, weight_k)

    plt.plot(arange(0,cnt),w_peak_k,'-k')

    title(fname, fontsize=8)
    xlabel('$k$',fontsize=17)
    ylabel('1A1_lowpeak_E',fontsize=17)
    xticks((0,Nk,2.*Nk,3.*Nk), (r'$\Gamma$','M','X',r'$\Gamma$'))
    #text(0.45, 0.1, '(a)', fontsize=16)
    grid('on',linestyle="--", linewidth=0.5, color='black', alpha=0.5)
    legend(loc='best', fontsize=9.5, framealpha=1.0, edgecolor='black')
    savefig("1A1_lowpeak_E_dispersion_"+fname+".pdf")

    clf()
    plt.plot(arange(0,cnt),weight_k,'-k')

    title(fname, fontsize=8)
    xlabel('$k$',fontsize=17)
    ylabel('1A1_lowpeak_weight',fontsize=17)
    xticks((0,Nk,2.*Nk,3.*Nk), (r'$\Gamma$','M','X',r'$\Gamma$'))
    #text(0.45, 0.1, '(a)', fontsize=16)
    grid('on',linestyle="--", linewidth=0.5, color='black', alpha=0.5)
    legend(loc='best', fontsize=9.5, framealpha=1.0, edgecolor='black')
    savefig("1A1_lowpeak_weight_dispersion_"+fname+".pdf")

Populating the interactive namespace from numpy and matplotlib
Cu_orbs =  ['dx2y2']
O1_orbs =  ['px']
O2_orbs =  ['py']
Mc= 160
k =  0.0 0.0
VS.dim =  232003
No. of states with count_upup, count_updn, count_dnup, count_dndn: 58000 58003 58000 58000
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_edep_diag_matrix
dd_state_indices 0 , state:  up dx2y2 dn dx2y2
peak w-range = [ 0.48750000000006377 0.9750000000000707 ]
peak, weight =  0.7250000000000671    0.026947709237694132
k =  0.7853981633974483 0.7853981633974483
VS.dim =  232003
No. of states with count_upup, count_updn, count_dnup, count_dndn: 58000 58003 58000 58000
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_edep_diag_matrix


KeyboardInterrupt: 